In [208]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 1000
pd.set_option('max_colwidth', -1)
import requests
from IPython.display import HTML


/var/folders/cg/b2_z5vjn52x7nyrtvtmwqsqh0000gp/T/ipykernel_30115/2369163326.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


# RGE

Les données RGE se trouvent sur le portail data.ademe.fr ici : https://data.ademe.fr/datasets/liste-des-entreprises-rge-2
On peut y accéder via une api (assez lente) par chunk de X lignes. J'ai choisi ci-dessous des chunk de 10 000 lignes, il y a 210 000 lignes dans le fichier cible, ce qui fait 21 chunk.

In [113]:
arr = []
r = requests.get('https://data.ademe.fr/data-fair/api/v1/datasets/liste-des-entreprises-rge-2/lines?size=10000&select=siret%2Ccode_qualification')
data = r.json()
arr = arr + data['results']

In [114]:
cpt = 0
while('next' in data):
    cpt = cpt + 1
    if(cpt % 5 == 0):
        print(cpt)
    r = requests.get(data['next'])
    data = r.json()
    arr = arr + data['results']

5
10
15
20


In [115]:
df = pd.DataFrame(arr)

In [116]:
df['siren'] = df['siret'].str[:9]

In [117]:
res = df.groupby(['siren'])['code_qualification'].apply(list).reset_index(name="liste_code_qualification_rge")

In [118]:
res['is_rge'] = True

In [119]:
res.head(10)

,siren,liste_code_qualification_rge,is_rge
0,000000000,"[2, NFMR440, AUDIT500-02, NFMR440, AUDIT500-02, NFMR440, AUDIT500-02, NFMR440, NFMR440, NFMR440, NFMR440, NFMR440, AUDIT500-02, NFMR440, AUDIT500-02, NFMR440, AUDIT500-02, NFMR440, NFMR440, NFMR440]",True
1,000000001,"[3511D109, 3521D109, 3222D113, 3511D109, 3511D109, 3511D109, 3521D109, 4522D109, 3211D113, 5221D104, 3511D109, 3511D109, 3511D109, 7131D112, 7131D115, 4131D111, 3101D110, 3101D111, 3101D112, 3101D113, 3101D114, 3511D109, 3211D113, 3511D109, 3511D109, 4511D109, 3511D109, 3511D109, 3511D109, 3511D109, 5211D101, 5211D107, 5221D104, 5221D104]",True
2,006076434,"[8611M3D111, 8611M3D112, 8611M3D115, 8611M4D111, 8611M4D114, 8611M4D115, 8611M10D109]",True
3,006850259,[3511D109],True
4,006980395,"[8611M3D109, 8611M3D111, 8611M3D112, 8611M3D114, 8611M4D111, 8611M4D114, 8611M10D109]",True
5,007050123,[21],True
6,007080229,[7122D114],True
7,007280472,"[8611M3D111, 8611M3D112, 8611M3D113, 8611M3D114, 8611M3D115, 8611M10D109]",True
8,007350184,"[5211D101, 8621T16D111, 8621T16D112, 8621T16D115, 8621T17D110, 8621T17D111, 8621T17D113, 8621T17D114, 8621T18D109, 8621T18D110, 23, 41, 43, 22, 11]",True
9,015551401,[3511D109],True


ces codes qualifications permettent de faire la jointure avec leurs noms et leurs domaines. Pour ne pas surcharger l'index es, je propose de ne pas indexer ces données. Par contre, je les ai stocker dans un fichier csv ici (qui pourra servir au front pour faire pivot) : 

In [149]:
ref = pd.read_csv('./data/ref_qualification_rge.csv',dtype=str)

In [150]:
ref.head(2)

,code_qualification,nom_qualification,meta_domaine
0,43,QualiPAC Chauffage - Pose de chauffe-eau thermodynamique (43),Installations d'énergies renouvelables
1,41,QualiPAC Chauffage - Pose de pompe à chaleur (PAC aérothermique ou géothermique et chauffe-eau thermodynamique) (41),Installations d'énergies renouvelables


A voir plus si on veut également indexer le "meta-domaine". Dans un premier temps pas utile je pense

In [122]:
HTML('<img src="https://media.giphy.com/media/3o7qDSOvfaCO9b3MlO/giphy-downsized.gif">')

# Entrepreneurs de spectacles vivant

In [123]:
r = requests.get('https://www.data.gouv.fr/fr/datasets/r/fb6c3b2e-da8c-4e69-a719-6a96329e4cb2')
with open('./data/entrepreneurs-spectacles-vivant.csv', 'wb') as f:
    for chunk in r.iter_content(1024):
        f.write(chunk)

In [124]:
df = pd.read_csv('./data/entrepreneurs-spectacles-vivant.csv',dtype=str,sep=";")

In [125]:
# btw mon coup de coeur compagnie de théâtre :
df[df['raison_sociale_personne_morale_ou_nom_personne_physique'].str.contains('SI VOUS POUVIEZ')]

,numero_de_recepisse,date_de_depot_de_la_declaration_inscrite_sur_le_recepisse,statut_du_recepisse,categorie,type,date_de_validite_du_recepisse_sauf_opposition_de_l_administration,type_de_declarant,raison_sociale_personne_morale_ou_nom_personne_physique,code_postal_de_l_etablissement_principal_personne_morale_ou_de_la_personne_physique,siren_personne_physique_siret_personne_morale,code_naf_ape,nom_du_lieu,code_postal_du_lieu,geoloc_cp,region,departement
26466,PLATESV-R-2021-001389,2021-02-04,Valide,2,Renouvellement,06/03/2021,Personne morale,COLLECTIF SI VOUS POUVIEZ LECHER MON COEUR,62100,75093029900029,90.01Z - Arts du spectacle vivant,NaN,NaN,"50.9500651474,1.87611226111",Hauts-de-France,Pas-de-Calais


On ne garde que les licences valides :

In [126]:
df  = df[df['statut_du_recepisse'] == 'Valide']

In [127]:
df['is_entrepreneur_spectacle'] = True

In [128]:
df['siren'] = df['siren_personne_physique_siret_personne_morale'].str[:9]

In [129]:
df = df[['siren', 'is_entrepreneur_spectacle']]

In [130]:
df.head(10)

,siren,is_entrepreneur_spectacle
194,453710956,True
195,845392729,True
196,882508757,True
197,515076933,True
198,490274933,True
199,482257011,True
200,877698431,True
201,514634658,True
202,883565921,True
237,824496350,True


A indexer tel quel !

In [131]:
HTML('<img src="https://media.giphy.com/media/26tP8qAV3GyP1Pdde/giphy-downsized.gif">')

# Numéro UAI / Annuaire de l'éducation

In [132]:
r = requests.get('https://www.data.gouv.fr/fr/datasets/r/b22f04bf-64a8-495d-b8bb-d84dbc4c7983')
with open('./data/annuaire-education.csv', 'wb') as f:
    for chunk in r.iter_content(1024):
        f.write(chunk)

In [133]:
df = pd.read_csv('./data/annuaire-education.csv',dtype=str,sep=";")

In [134]:
df = df[['identifiant_de_l_etablissement', 'siren_siret', 'code_nature']]

In [135]:
df = df.rename(columns={'identifiant_de_l_etablissement': 'uai', 'siren_siret': 'siren'})

In [136]:
df['siren'] = df['siren'].str[:9]

In [137]:
res = df.groupby(['siren'])['uai'].apply(list).reset_index(name="liste_uai")

In [138]:
res2 = df.groupby(['siren'])['code_nature'].apply(list).reset_index(name="liste_code_nature")

In [139]:
res = res.merge(res2,on='siren',how='left')

In [140]:
res['liste_code_nature'] = res['liste_code_nature'].apply(lambda x: list(set(x)))

In [ ]:
res['is_etablissement_education'] = True

In [141]:
# Exemple ville de paris :
res[res['siren'] == '217500016']

,siren,liste_uai,liste_code_nature
25621,217500016,"[0750602X, 0750810Y, 0750820J, 0750825P, 0750826R, 0750841G, 0750843J, 0750855X, 0750857Z, 0750862E, 0750863F, 0750949Z, 0750954E, 0750956G, 0750962N, 0750965S, 0750969W, 0750978F, 0750982K, 0750984M, 0751019A, 0751023E, 0751029L, 0751032P, 0751038W, 0751054N, 0751072H, 0751074K, 0751076M, 0751098L, 0751100N, 0751102R, 0751106V, 0751109Y, 0751116F, 0751122M, 0751128U, 0751129V, 0751139F, 0751144L, 0751175V, 0751177X, 0751178Y, 0751179Z, 0751191M, 0751200X, 0751212K, 0751215N, 0751216P, 0751225Z, 0751247Y, 0751250B, 0751254F, 0751273B, 0751284N, 0751288T, 0751290V, 0751298D, 0751301G, 0751313V, 0751314W, 0751315X, 0751342B, 0751336V, 0751354P, 0751357T, 0751358U, 0751378R, 0751381U, 0751383W, 0751385Y, 0751391E, 0751392F, 0751398M, 0751409Z, 0751411B, 0751415F, 0751429W, 0751726U, 0751728W, 0752213Y, 0752216B, 0752261A, 0752263C, 0752264D, 0752265E, 0752266F, 0752328Y, 0752405G, 0752410M, 0752565F, 0752569K, 0752573P, 0752574R, 0752622T, 0752636H, 0752763W, 0752770D, 0752835Z, 0752839D, ...]","[103, 151, 101, 320, 153]"


In [143]:
res.head(5)

,siren,liste_uai,liste_code_nature,is_etablissement_education
0,024053852,[9760278Y],[151],True
1,057817314,"[0131796G, 0131335F, 0132923G]","[302, 151, 340]",True
2,058806100,"[0132913W, 0131323T, 0132038V]","[302, 151, 340]",True
3,069501500,"[0382579X, 0380417X]","[151, 340]",True
4,069502052,"[0381957W, 0382581Z, 0381803D]","[302, 151, 340]",True


ces codes nature permettent de faire la jointure avec le type d'établissement. Pour ne pas surcharger l'index es, je propose de ne pas indexer ces données. Par contre, je les ai stocker dans un fichier csv ici (qui pourra servir au front pour faire pivot) :

In [151]:
ref = pd.read_csv('./data/ref_code_nature_education.csv',dtype=str)

In [152]:
ref.head(5)

,code_nature,libelle_nature
0,151,ECOLE DE NIVEAU ELEMENTAIRE
1,101,ECOLE MATERNELLE
2,153,ECOLE ELEMENTAIRE D APPLICATION
3,170,ECOLE SANS EFFECTIFS PERMANENTS
4,103,ECOLE MATERNELLE D APPLICATION


In [144]:
HTML('<img src="https://media.giphy.com/media/d0NnEG1WnnXqg/giphy-downsized.gif">')

# Numéro FINESS - Etablissements sanitaires et social

In [145]:
r = requests.get('https://www.data.gouv.fr/fr/datasets/r/2ce43ade-8d2c-4d1d-81da-ca06c82abc68')
with open('./data/annuaire-finess.csv', 'wb') as f:
    for chunk in r.iter_content(1024):
        f.write(chunk)

In [234]:
df = pd.read_csv(
    './data/annuaire-finess.csv',
    dtype=str,
    sep=";",
    encoding="Latin-1",
    skiprows=1,
    header=None
)

In [235]:
df = df[[1,18,22]]

In [236]:
df = df.rename(columns={1: 'finess', 18: 'cat_etablissement', 22: 'siren'})

In [237]:
df['siren'] = df['siren'].str[:9]

In [238]:
df = df[df['siren'].notna()]

In [239]:
res = df.groupby(['siren'])['finess'].apply(list).reset_index(name="liste_finess")

In [240]:
res2 = df.groupby(['siren'])['cat_etablissement'].apply(list).reset_index(name="liste_cat_etablissement")

In [241]:
res = res.merge(res2, on='siren', how='left')

In [242]:
res['liste_cat_etablissement'] = res['liste_cat_etablissement'].apply(lambda x: list(set(x)))

In [243]:
res['is_finess'] = True

In [244]:
# exemple Croix rouge française :
res[res['siren'] == '775672272']

,siren,liste_finess,liste_cat_etablissement,is_finess
31182,775672272,"[010780799, 010784130, 010788503, 020004438, 020008629, 020010898, 030782924, 060019809, 060020427, 060021524, 060781176, 060782000, 060782299, 060787553, 060790425, 060792314, 070783204, 080000383, 080003049, 080003312, 080005424, 080005705, 080007370, 090003054, 090003526, 090003997, 090004391, 100001908, 100002252, 100010529, 100011279, 130021538, 130045487, 130047517, 130051063, 130783749, 130789514, 130797640, 130807043, 140002619, 140008202, 140016957, 140017146, 140025099, 140030198, 140033184, 140033192, 140033200, 150781888, 160004271, 160004784, 160007357, 170780035, 170780043, 180003618, 190001677, 190011262, 210001434, 210001442, 210004909, 210008108, 210008298, 210009981, 210970091, 210984159, 210984852, 210986956, 240006742, 240014274, 250008448, 250019544, 260004650, 260005111, 260005640, 270008766, 270011117, 270013618, 270025208, 290015650, 290032481, 300004579, 300011574, 300013075, 300017506, 300017514, 300018066, 300019296, 300780384, 300783537, 300784014, 300784238, 300784675, 300786340, 300788015, 310022173, 310023270, 310786439, 320784622, 330023508, 330058900, ...]","[165, 271, 448, 207, 214, 247, 365, 443, 189, 175, 340, 230, 221, 209, 449, 442, 379, 436, 500, 380, 180, 177, 382, 460, 445, 370, 228, 255, 381, 378, 219, 192, 698, 109, 197, 266, 446, 178, 124, 127, 330, 183, 354, 258, 300, 202, 246, 161, 437, 166, 182, 188, 236, 238, 252, 259, 223]",True


In [245]:
res.head(5)

,siren,liste_finess,liste_cat_etablissement,is_finess
0,005720164,[800002503],[365],True
1,006640668,[040003022],[620],True
2,007050040,"[040002248, 040002289, 040780405]","[500, 109, 362]",True
3,007050073,[040780488],[109],True
4,015550353,[210005153],[620],True


ces catégories établissements permettent de faire la jointure avec le type d'établissement. Pour ne pas surcharger l'index es, je propose de ne pas indexer ces données. Par contre, je les ai stocker dans un fichier csv ici (qui pourra servir au front pour faire pivot) :

In [248]:
ref = pd.read_csv('./data/ref_categorie_etablissement_sanitaire_social.csv', dtype=str)

In [249]:
ref.head(5)

,cat_etablissement,libelle_cat_etablissement
0,355,Centre Hospitalier (C.H.)
1,106,"Centre hospitalier, ex Hôpital local"
2,292,Centre Hospitalier Spécialisé lutte Maladies Mentales
3,202,Résidences autonomie
4,246,Etablissement et Service d'Aide par le Travail (E.S.A.T.)


In [250]:
HTML('<img src="https://media.giphy.com/media/Z6DPKw6gyCYz4zEsxk/giphy.gif">')

# Organismes de formation

In [189]:
r = requests.get('https://www.data.gouv.fr/fr/datasets/r/ac59a0f5-fa83-4b82-bf12-3c5806d4f19f')
with open('./data/organismes-formation.csv', 'wb') as f:
    for chunk in r.iter_content(1024):
        f.write(chunk)
        

In [267]:
df = pd.read_csv('./data/organismes-formation.csv', dtype=str, sep=";")

In [268]:
df

,numeroDeclarationActivite,numerosDeclarationActivitePrecedent,denomination,siren,siretEtablissementDeclarant,adressePhysiqueOrganismeFormation.voie,adressePhysiqueOrganismeFormation.codePostal,adressePhysiqueOrganismeFormation.ville,adressePhysiqueOrganismeFormation.codeRegion,certifications.actionsDeFormation,certifications.bilansDeCompetences,certifications.VAE,certifications.actionsDeFormationParApprentissage,organismeEtrangerRepresente.denomination,organismeEtrangerRepresente.voie,organismeEtrangerRepresente.codePostal,organismeEtrangerRepresente.ville,organismeEtrangerRepresente.pays,informationsDeclarees.dateDerniereDeclaration,informationsDeclarees.debutExercice,informationsDeclarees.finExercice,informationsDeclarees.specialitesDeFormation.codeSpecialite1,informationsDeclarees.specialitesDeFormation.libelleSpecialite1,informationsDeclarees.specialitesDeFormation.codeSpecialite2,informationsDeclarees.specialitesDeFormation.libelleSpecialite2,informationsDeclarees.specialitesDeFormation.codeSpecialite3,informationsDeclarees.specialitesDeFormation.libelleSpecialite3,informationsDeclarees.nbStagiaires,informationsDeclarees.nbStagiairesConfiesParUnAutreOF,informationsDeclarees.effectifFormateurs,Unnamed: 30
0,11788457578,11788457578,01 AFFORP,890176654,89017665400014,NaN,NaN,NaN,11,true,false,false,false,NaN,NaN,NaN,NaN,NaN,22/04/2022,01/11/2021,31/12/2022,100,Formations générales,312,"Commerce, vente",NaN,NaN,NaN,NaN,5,NaN
1,11930884493,11930884493,093 LAB,848912317,84891231700023,NaN,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26/05/2022,01/01/2022,31/12/2022,134,Autres disciplines artistiques et spécialités artistiques plurivalentes,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,98970223197,98970223197,0CEAN INDIEN INFORMATIQUE SARL,384520045,38452004500046,NaN,NaN,NaN,04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12/05/2022,31/12/2021,30/12/2022,310,Spécialités plurivalentes des échanges et de la gestion,326,"Informatique, traitement de l'information, réseaux de transmission des données",333,"Enseignement, formation",178,0,2,NaN
3,02973363297,02973363297,0'LISTY,889434254,88943425400013,16 avenue Nelson Mandela,97227,STE ANNE,02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03/03/2022,01/10/2021,30/09/2022,999,Autres...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,27210423521,27210423521,1000 BORNES,853070209,85307020900020,27 RUE DE LA LIBERTE,21500,MONTBARD,27,true,false,false,false,NaN,NaN,NaN,NaN,NaN,01/07/2021,01/06/2021,31/05/2022,333,"Enseignement, formation",NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119843,11941095194,11941095194,ZYC FORMATION,904301520,90430152000014,137 AV ANATOLE FRANCE,94600,CHOISY LE ROI,11,true,false,false,false,NaN,NaN,NaN,NaN,NaN,08/12/2021,18/10/2021,17/10/2022,310,Spécialités plurivalentes des échanges et de la gestion,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
119844,32800223780,32800223780,ZY LEARNING,914089792,91408979200016,23 RUE DE METZ,80000,AMIENS,32,true,false,false,false,NaN,NaN,NaN,NaN,NaN,07/07/2022,24/05/2022,31/12/2022,414,Développement des capacités individuelles d'organisation,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
119845,82740033374,82740033374,ZYLIA TECH,353652787,35365278700022,NaN,NaN,NaN,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05/05/2022,01/01/2022,31/12/2022,201,Technologies de commandes des transformations industrielles,NaN,NaN,NaN,NaN,24,0,2,NaN
119846,11755090675,11755090675,Z. YOUAKIM,539019943,53901994300049,NaN,NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26/05/2022,01/01/2022,31/12/2022,330,Spécialités plurivalentes sanitaires et sociales,413,Développement des capacités comportementales et relationnelles,NaN,NaN,31,0,1,NaN


In [269]:
df = df[['numeroDeclarationActivite', 
         'siren',
         'siretEtablissementDeclarant',
         'certifications.actionsDeFormation',
         'certifications.bilansDeCompetences', 
         'certifications.VAE',
         'certifications.actionsDeFormationParApprentissage',
         'informationsDeclarees.specialitesDeFormation.codeSpecialite1',
         'informationsDeclarees.specialitesDeFormation.codeSpecialite2',
         'informationsDeclarees.specialitesDeFormation.codeSpecialite3'
]]

In [270]:
df = df.rename(columns={
         'numeroDeclarationActivite': 'id_of', 
         'siretEtablissementDeclarant': 'siret',
         'certifications.actionsDeFormation': 'cert_actions_formation',
         'certifications.bilansDeCompetences': 'cert_bilans_competences', 
         'certifications.VAE': 'cert_vae',
         'certifications.actionsDeFormationParApprentissage': 'cert_actions_formation_apprentissage',
         'informationsDeclarees.specialitesDeFormation.codeSpecialite1': 'code_spe1',
         'informationsDeclarees.specialitesDeFormation.codeSpecialite2': 'code_spe2',
         'informationsDeclarees.specialitesDeFormation.codeSpecialite3': 'code_spe3'
})

In [274]:
df

,id_of,siren,siret,cert_actions_formation,cert_bilans_competences,cert_vae,cert_actions_formation_apprentissage,code_spe1,code_spe2,code_spe3
0,11788457578,890176654,89017665400014,true,false,false,false,100,312,NaN
1,11930884493,848912317,84891231700023,NaN,NaN,NaN,NaN,134,NaN,NaN
2,98970223197,384520045,38452004500046,NaN,NaN,NaN,NaN,310,326,333
3,02973363297,889434254,88943425400013,NaN,NaN,NaN,NaN,999,NaN,NaN
4,27210423521,853070209,85307020900020,true,false,false,false,333,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
119843,11941095194,904301520,90430152000014,true,false,false,false,310,NaN,NaN
119844,32800223780,914089792,91408979200016,true,false,false,false,414,NaN,NaN
119845,82740033374,353652787,35365278700022,NaN,NaN,NaN,NaN,201,NaN,NaN
119846,11755090675,539019943,53901994300049,NaN,NaN,NaN,NaN,330,413,NaN


In [275]:
df['code_spe'] = df[["code_spe1", "code_spe2", "code_spe3"]].values.tolist()

In [276]:
df['code_spe'] = df['code_spe'].apply(lambda x: [v for v in x if v == v])

In [278]:
df = df[['id_of', 'siren', 'siret', 'cert_actions_formation',
       'cert_bilans_competences', 'cert_vae',
       'cert_actions_formation_apprentissage', 'code_spe']]

In [279]:
df

,id_of,siren,siret,cert_actions_formation,cert_bilans_competences,cert_vae,cert_actions_formation_apprentissage,code_spe
0,11788457578,890176654,89017665400014,true,false,false,false,"[100, 312]"
1,11930884493,848912317,84891231700023,NaN,NaN,NaN,NaN,[134]
2,98970223197,384520045,38452004500046,NaN,NaN,NaN,NaN,"[310, 326, 333]"
3,02973363297,889434254,88943425400013,NaN,NaN,NaN,NaN,[999]
4,27210423521,853070209,85307020900020,true,false,false,false,[333]
...,...,...,...,...,...,...,...,...
119843,11941095194,904301520,90430152000014,true,false,false,false,[310]
119844,32800223780,914089792,91408979200016,true,false,false,false,[414]
119845,82740033374,353652787,35365278700022,NaN,NaN,NaN,NaN,[201]
119846,11755090675,539019943,53901994300049,NaN,NaN,NaN,NaN,"[330, 413]"


In [289]:
df[df['siren'] == '200076958']

,id_of,siren,siret,cert_actions_formation,cert_bilans_competences,cert_vae,cert_actions_formation_apprentissage,code_spe
61797,94202036120,200076958,20007695800053,NaN,NaN,NaN,NaN,[221]
61799,9420P107720,200076958,20007695800012,true,false,false,false,[221]


Faire comme pour les dirigeants :
- insérer tel quel dans sqlite (attention à la liste)
- dans la requête es : faire un requêtage de type json pour récupérer toutes les lignes correspondant à un siret.
Ex pour siren 200076958 :
```
[
  {
    "id_of": "94202036120"
    "siret": "20007695800053",
    "cert_...": ...,
    "code_spe": ...
  },
  {
    "id_of": "9420P107720",
    "siret": "20007695800012",
    "cert...": ...,
    "code_spe": ...
  }
]
```

!!! Faire une propriété `is_of` à True pour l'ensemble de ces siren

ces codes spécialités permettent de faire la jointure avec le libellé de la spécialité. Pour ne pas surcharger l'index es, je propose de ne pas indexer ces données. Par contre, je les ai stocker dans un fichier csv ici (qui pourra servir au front pour faire pivot) :

In [292]:
ref = pd.read_csv('./data/ref_code_specialite_formation.csv',dtype=str)

In [294]:
ref.head(5)

,code_specialite,libelle_specialite
0,100,Formations générales
1,134,Autres disciplines artistiques et spécialités artistiques plurivalentes
2,310,Spécialités plurivalentes des échanges et de la gestion
3,999,Autres...
4,333,"Enseignement, formation"


In [295]:
HTML('<img src="https://media.giphy.com/media/XKCdA6ERnXp6M/giphy.gif">')